In [ ]:
import numpy as np
import librosa
import IPython.display as ipd

from glob import glob
from scipy.io import loadmat

import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import os


In [ ]:
audio_s = "/data/datasets/ai_challenge/ST_attention_Libri_aurora/0/car/wav/103-1240-0000-car.wav"
audio_label = "/data/datasets/ai_challenge/ST_attention_Libri_aurora/train-clean-100-label-rvad-ext/103-1240-0000-rvad-ext.npy"

d_ = librosa.load(audio_s, sr=16000)
l_ = np.load(audio_label)
d_.shape, l_.shape

## 데이터 파악
### 데이터를 로드하면 'Signal_0' 그리고 'Signal_1' 구조가 존재 함.
#### Signal_0 : d, 사람이 듣는 신호에 대한 신호 8 채널 데이터로 추정
#### Signal_1 : Y, 추론에 필요한 가속도계 신호인 66채널의 데이터로 추정
####           => 10, 11, 12, 28, 29, 30 46, 47, 48, 61, 62, 63 채널을 사용할 것

In [2]:
base_path = "/data/datasets/hyundai"

main_data_path = os.path.join(base_path, "data")
main_mats = glob(main_data_path + "/*.mat")

sample_data_path = os.path.join(base_path, "20200619_DNN_ANC_sample_secondary_path")
sample_mats = glob(sample_data_path + "/*.mat")

S_mat_path = "/data/datasets/hyundai/20200619_DNN_ANC_sample_secondary_path/S_128_8_8.mat"
S_mat = loadmat(S_mat_path, mat_dtype=True)

S_data = S_mat['S']

In [3]:
acceleration_chans = [10, 11, 12, 28, 29, 30, 46, 47, 48, 61, 62, 63]
acceleration_data = [] # 차체에서 수집한 가속도계 데이터
sound_data = [] # 사람이 듣는 위치에서 녹음한 소리 신호 정보

In [5]:
for i, mat in tqdm(enumerate(main_mats)):
    print(mat)
    mat_data = loadmat(mat, mat_dtype=True)
    
    if mat_data['Signal_0'][0,0]['y_values']['values'][0,0].shape[-1] > 8:
        sig_sound = mat_data['Signal_1']
        sig_accel = mat_data['Signal_0']
    else:
        sig_sound = mat_data['Signal_0']
        sig_accel = mat_data['Signal_1']
    
    value_sound = sig_sound[0,0]['y_values']['values'][0,0][:,:]
    value_sound = np.array(value_sound)
    print("value sound: ", value_sound.shape)
    sound_data.append(value_sound)
    
    value_accel = sig_accel[0,0]['y_values']['values'][0,0][:, acceleration_chans]
    value_accel = np.array(value_accel)
    acceleration_data.append(value_accel)
    print("acceleration data: ", value_accel.shape)

0it [00:00, ?it/s]

/data/datasets/hyundai/data/Univ_Rough_60kph_1.mat


2it [00:00,  1.73it/s]

value sound:  (177398, 8)
acceleration data:  (177398, 12)
/data/datasets/hyundai/data/Univ_Rough_80kph_1.mat
value sound:  (149702, 8)
acceleration data:  (149702, 12)
/data/datasets/hyundai/data/Univ_Rough_70kph_1.mat


3it [00:01,  1.90it/s]

...

value sound:  (234512, 8)
acceleration data:  (234512, 12)
/data/datasets/hyundai/data/Univ_Rough_60kph_2.mat


5it [00:02,  2.09it/s]

value sound:  (224472, 8)
acceleration data:  (224472, 12)
/data/datasets/hyundai/data/Univ_Rough_50kph_1.mat


6it [00:02,  2.51it/s]

value sound:  (234880, 8)
acceleration data:  (234880, 12)


In [6]:
sound_save_path = os.path.join(base_path, "sound_data.pickle")
with open(sound_save_path, 'wb') as f:
    pickle.dump(sound_data, f)

accel_save_path = os.path.join(base_path, "accel_data.pickle")
with open(accel_save_path, 'wb') as f:
    pickle.dump(acceleration_data, f)

In [11]:
listen = sound_data[0][:,0]
listen.shape

(177398,)

In [17]:
ipd.Audio(listen, rate=16000)